In [3]:
import MDAnalysis as mda
from MDAnalysis import transformations
import numpy as np

import os
import shutil

Importing the modules needed to move protein. 'edit_box' folder should be in working directory

In [4]:
from edit_box import move_prot

Calling move_prot function. Should have 2 folders:
- amino acid folder with topologies and gro file
- LD file with topologies and gro file
- eq file that has mdp options

We are making 'out' file which has final data

In [6]:
os.mkdir('out')

function below: ( LD gro file, amino acid gro file, # angstroms above membrane, out file path )

In [7]:
combined = move_prot.move_prot('ld_top/ld0.gro', 'phe/1.gro', 20, 'out/out.gro')

Gromacs has to be loaded in terminal before calling this. This renumbers gro file. If this is on CHPC, then gmx_mpi

In [8]:
!gmx genconf -renumber -f out/out.gro -o out/renumber.gro


                      :-) GROMACS - gmx genconf, 2022 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /mnt/c/users/jay/desktop/modules/modules/move_prot
Command line:
  gmx genconf -renumber -f out/out.gro -o out/renumber.gro


GROMACS reminds you: "Computer dating is fine, if you are a computer." (Rita May Brown)



Copy the toppar from 'ld_top' to 'out'

In [13]:
in_path = 'ld_top/toppar'
out_path = 'out/toppar'
shutil.copytree(in_path, out_path) 

'out/toppar'

In [16]:
in_path = 'ld_top/topol_tg.top'
out_path = 'out/topol.top'
shutil.copy(in_path, out_path) 

'out/topol.top'

We are now moving the amino acid topology file (LIG.itp) from amino acid /toppar/ file to 'out' /toppar file.

In [14]:
inpath = r'phe/toppar/LIG.itp'
outpath = r'out/toppar/LIG.itp'
shutil.copy(inpath, outpath)

'out/toppar/LIG.itp'

In [17]:
## only use if old protein name is in there. If topology_tg file is coming from another 
## amino acid LD system, than it should be fine

## in development: add attribute thatchecks for name and if not there will add, or skip

oldname = 'PROA'
ligname = 'LIG'
with open(r'out/topol.top','r') as f:
    data = f.read()
    data = data.replace(oldname, ligname)
with open(r'out/topol.top','w') as f:
    f.write(data)
    

We want to count the number of water atoms in the new file. This is because the amino acids / ligand replaces the water, so we need to change the number in the topology top file

In [18]:
name = 'TIP3'
ls = []
with open(r'out/renumber.gro','r') as f:
    content = f.read()
    occurrences = content.count('TIP3')
    num = occurrences/3

Channging it to get rid of '.0' at end

In [19]:
num=int(num)

In [20]:
num=str(num)


In [21]:
num

'42364'

In [22]:
with open(r'out/topol.top','r') as s:
    lines = s.readlines()
    
    ls=[]
    for stuff in lines:
        if stuff.startswith('TIP3 '):
            ls.append(stuff)
            
            

In [23]:
ls

['TIP3  \t       41851\n']

In [25]:
# variable naming got sloppy here, should eventually change

file = open(r'out/topol.top','r')
data1 = file.read()
data2 = 'TIP3'+'          '+num
new_data = open(r'out/topol.top','r')
new_data1 = new_data.read()
new_data1 = new_data1.replace(stuff, data2)

with open(r'out/topol.top','w') as f:
    f.write(new_data1)

After this we have an amino-acid + LD system that is renumbered for the waters. At this point you should run 'gmx grompp' with the step6.0_minimization file. This will likely pop up and error saying that there is a + or - charge ibalance in the system 

Lets say for example we have an overall -1.0 (negative) charge.

 The ending of the renumber.gro will look something like this:


In [27]:
#...

#43318TIP3    OH235006   7.086   6.321  21.623  0.0000  0.0000  0.0000
#43318TIP3    H135007   7.090   6.330  21.527  0.0000  0.0000  0.0000
#43318TIP3    H235008   7.119   6.233  21.640  0.0000  0.0000  0.0000
#    9.56090   9.56090  25.41698

We will then copy a line from this same file that has SOD, and paste it at the end of the topology file as shown below:

In [ ]:
#...

#43318TIP3    OH235006   7.086   6.321  21.623  0.0000  0.0000  0.0000
#43318TIP3    H135007   7.090   6.330  21.527  0.0000  0.0000  0.0000
#43318TIP3    H235008   7.119   6.233  21.640  0.0000  0.0000  0.0000
#43319SOD     SOD35009   2.532   8.193   6.570  0.0000  0.0000  0.0000
#    9.56090   9.56090  25.41698

This requires you to add the numbering in front of the residue name and atom name (43319SOD     SOD35009) and also to change the coordinates slightly (2.532   8.193   6.570). If this was coppied from previous SOD line, just change the coordinates by +- 1 or so. 

You then have to change the number of atoms on the top of the topology file. Below says '235008', but you will have to change it to 235009

In [ ]:
#Written by MDAnalysis
#235008
#    1LIG     N1    1   4.691   4.743   4.661  0.0000  0.0000  0.0000
#    1LIG     C1    2   4.766   4.859   4.698  0.0000  0.0000  0.0000
#    1LIG     C2    3   4.768   4.960   4.588  0.0000  0.0000  0.0000
#    1LIG     O1    4   4.781   4.929   4.470  0.0000  0.0000  0.0000

From here try to minimize the system again with this new settup